### Graph Structures of TensorFlow

In [1]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import numpy as np


anchors       = np.array(((0.8224980405412367, 0.8422009361658516),
                         (0.3996183458965756, 0.7135638022627926),
                         (0.3124547970691455, 0.356975225778714),
                         (0.715922252069278, 0.47470965095772916),
                         (0.1272335914117044, 0.19679513461111448)))

n_anchors = len(anchors)
n_classes = 20
batch     = 2

# Example output layer of YOLOv2 in Batch
feature_map      = K.random_uniform_variable(shape=(batch, 3, 4, n_anchors, n_classes+5),low=0, high=1)
feature_map_dims = K.shape(feature_map)[1:3]

# Examples of ground truth values
ground_truth     = K.random_uniform_variable(shape=(batch, n_classes+5), low=0, high = 1)

# Anchors
anchors_tensor = K.reshape(K.variable(anchors), [1, 1, 1, n_anchors, 2])


In [ ]:
tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
print("Ground truth boxes: [batch_size, grid_w*grid_h, box_params]")
print(ground_truth.eval().shape)

In [ ]:
print("Predicted boxes: [batch_size, height, width, n_anchors, 5 + n_classes]")
print(feature_map.eval().shape)

In [ ]:
pred_shape = 3, 4
N_ANCHORS = 5
cx = K.arange(0, stop=pred_shape[0])
cx = K.tile(cx, [pred_shape[1]])
cy = K.arange(0, stop=pred_shape[1])
cy = K.tile(K.expand_dims(cy, 0), [pred_shape[0], 1])
cy = K.flatten(K.transpose(cy))

c_xy = K.transpose(K.stack([cx, cy]))
c_xy = K.reshape(c_xy, [1, pred_shape[1], pred_shape[0], 2])
c_xy = tf.transpose(c_xy, [0, 2, 1, 3])
c_xy = K.cast(c_xy, K.dtype(feats))
c_xy = K.expand_dims(c_xy, 3)
c_xy = K.tile(c_xy, [1, 1, 1, N_ANCHORS, 1])/ np.reshape([float(3), float(4)], [1, 1, 1, 1, 2])
print(cx.eval().shape)
print(cx.eval())

print(cy.eval().shape)
print(cy.eval())

print(c_xy.eval().shape)
print(c_xy.eval())

In [ ]:
t_xy = feature_map[..., :2]
print(t_xy.eval().shape)
print(t_xy.eval())

In [ ]:
b_xy = t_xy + c_xy
print(b_xy.eval())

In [ ]:
conv_shape   = K.shape(feature_map)[1:3]  # Get width, height - assuming 'channels_last' in Keras
feats        = K.reshape(feature_map, [-1, conv_shape[0], conv_shape[1], n_anchors, n_classes + 5])

cx = 0
cy = 0
# print(feature_map.eval())
box_xy          = K.sigmoid(feats[..., :2])
box_wh          = K.exp(feats[..., 2:4])
box_confidence  = K.sigmoid(feats[..., 4:5])
box_class_probs = K.softmax(feats[..., 5:])
print(box_xy.eval().shape)

In [ ]:
# batch, conv_height, conv_width, num_anchors, num_true_boxes, box_params
gt_shape = K.shape(ground_truth )
true_boxes = K.reshape(ground_truth, [gt_shape[0], 1, 1, 1, gt_shape[1], gt_shape[2]])
true_xy = true_boxes[..., 0:2]
true_wh = true_boxes[..., 2:4]

# Find IOU of each predicted box with each ground truth box.
true_wh_half = true_wh / 2.
true_mins = true_xy - true_wh_half
true_maxes = true_xy + true_wh_half



In [ ]:
# Anchors
cx = 0.
cy = 0.
px = anchors_tensor[..., 0]
py = anchors_tensor[...,1]
bx = K.sigmoid(feats[..., 0]) + cx
by = K.sigmoid(feats[..., 1]) + cy
bw = K.exp(feats[..., 2])*px
bh = K.exp(feats[..., 3])*py


In [ ]:
a = np.reshape([float(40), float(30)], [1, 1, 1, 1, 2])
print(a)